In [1]:
#Packages Needed for Analysis
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import statistics
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import warnings

In [2]:
xls = pd.ExcelFile('day_price_raw.xlsx') # Read In Full Excel File
frames = [] #Initialize list to hold each df per excel tab
tickers = [] #Initialize list to hold all the respective ticker names

In [3]:
for i in range(1,20): #Loop through numbers of tabs in excel spreadsheet
    df_temp = pd.read_excel(xls,'equity' + str(i)) #Create df for first each iteration
    ticker = df_temp.columns[1].split()[0] #Obtain ticker for respective stock
    new_header = df_temp.iloc[0] + '_' + ticker #Create a new header based on what stock we are analyzing
    tickers.append(ticker) #Add ticker variable into list, so we can keep all the information after loop finished
    df_temp.columns=new_header #Set the df based on updated headers with stock concatenation
    df_temp['Ticker'] = ticker #Create new column for ticker
    df_temp = df_temp.drop(df_temp.index[0]) #Drop indexed row 0 
    df_temp['Year'] = pd.DatetimeIndex(df_temp['Dates' + '_' + ticker]).year #Add a year column as an attribute
    df_temp['Month'] = pd.DatetimeIndex(df_temp['Dates'+ '_' + ticker]).month #Add a month column as an attribute
    df_temp['Yesterday_Close' + '_' + ticker] = df_temp['Last Price' + '_' + ticker].shift(+1) #Feature generation 
    df_temp['Delta Change' + '_' + ticker] = df_temp['Last Price' + '_' + ticker] - df_temp['Yesterday_Close' + '_' + ticker]
    df_temp['%_Gain' + '_' + ticker] = (df_temp['Delta Change' + '_' + ticker] / df_temp['Last Price' + '_' + ticker])*100
    df_temp.drop(columns = ['Dates'+ '_' + ticker],axis = 1, inplace = True) #Drop date column
    df_temp.drop(columns = ['Ticker'], axis = 1, inplace = True) #Drop ticker column, otherwise would need dummy variables
    frames.append(df_temp) #add df into list in order to merge into master df
df = pd.concat(frames, sort=False, axis = 1) #Concatenate all df into one master df
df = df.loc[:,~df.columns.duplicated()] #Remove duplicates year and month  features 
df.columns = df.columns.str.replace("INDU", "^DJI") #Yahoo Finance uses differenct ticker symbol for Dow Jones & SPX Indexes
df.columns = df.columns.str.replace("SPX", "^GSPC")
df.head() #Look at master df

,Low Price_CTAS,High Price_CTAS,Open Price_CTAS,Volume_CTAS,Last Price_CTAS,Year,Month,Yesterday_Close_CTAS,Delta Change_CTAS,%_Gain_CTAS,...,Delta Change_^DJI,%_Gain_^DJI,Low Price_^GSPC,High Price_^GSPC,Open Price_^GSPC,Volume_^GSPC,Last Price_^GSPC,Yesterday_Close_^GSPC,Delta Change_^GSPC,%_Gain_^GSPC
1,42.67,43.87,43.64,1391349,43.16,2005,1,NaN,NaN,NaN,...,NaN,NaN,1200.3,1217.9,1211.92,1343537689,1202.08,NaN,NaN,NaN
2,42.27,43.78,43.54,1053048,42.55,2005,1,43.16,-0.61,-1.43361,...,-98.65,-0.927966,1185.39,1205.84,1202.08,1565299619,1188.05,1202.08,-14.03,-1.18093
3,41.77,43.11,42.03,1031141,42.44,2005,1,42.55,-0.11,-0.259189,...,-32.95,-0.310913,1183.72,1192.75,1188.05,1437634094,1183.74,1188.05,-4.31,-0.3641
4,42.2,43.26,42.34,1595734,42.85,2005,1,42.44,0.41,0.956826,...,25.05,0.235812,1183.23,1191.63,1183.74,1341172449,1187.88,1183.74,4.14,0.34852
5,42.7,44.03,42.91,1734152,43.77,2005,1,42.85,0.92,2.1019,...,-18.92,-0.178424,1182.11,1192.2,1187.89,1283372484,1186.19,1187.88,-1.69,-0.142473


In [4]:
print(df.isnull().sum().sum()) #Look for null values -  these occur when we should our "last price" feature one day
df.dropna(how = 'any',inplace = True) #drop the rows with missing values
df.shape #3901 days of daily data, with features columns


57


(3901, 154)

In [25]:
#Change the ticker names in our list to match yahoo finance for web scraping purposes
print("Number of Stocks:",len(tickers))
tickers = tickers[:-2]
tickers.append('^DJI')
tickers.append('^GSPC')
tickers[0:5]

Number of Stocks: 19


['CTAS', 'WAT', 'CTL', 'IT', 'MHK']

In [6]:
#Perform regularization to try and reduce number of features. Using logistic regression with l1 penalty as proxy for lasso regression due to classification problem woth 0/1 labels
df_lasso = deepcopy(df) #Create a new copy of master df to avoid altering the master df accidentally
CTAS = 'CTAS' #Creating a variable to reference any column ticker
df_lasso['Next_Day_Close' + '_' + CTAS] = df_lasso['Last Price' + '_' + CTAS].shift(-1) #Want to predict next day stock price. Shifting back 1 day to create labels
df_lasso['Price Change' + '_' + CTAS] = df_lasso['Next_Day_Close' + '_' + CTAS] - df_lasso['Last Price' + '_' + CTAS] #Determine price change in stock prices
df_lasso['D_to_D_Gain' + '_' + CTAS] = (df_lasso['Price Change' + '_' + CTAS] / df_lasso['Last Price' + '_' + CTAS])*100 #Determine ercent change in stock prices
df_lasso['Invest' + '_' + CTAS] =  (df_lasso['D_to_D_Gain' + '_' + CTAS] > .35).astype(int) #Create 0/1 labels based on if stock price closed by more than 0.35% 
num_0 = df_lasso.groupby('Invest' + '_' + CTAS)['Next_Day_Close' + '_' + CTAS].count()[0] #Look at number of data points closed less than 0.35%
num_1 = df_lasso.groupby('Invest' + '_' + CTAS)['Next_Day_Close' + '_' + CTAS].count()[1] #Look at number of data points closed greater than  or equal to 0.35%
print(num_0,num_1) #Print out number of 0/1 rows (2419 0 (no) and 1481 1 (yes))
df_lasso.dropna(how = 'any',inplace = True) #Remove NA's
df_lasso.drop(columns = ['Price Change' + '_' + CTAS,'Next_Day_Close' + '_' + CTAS,'D_to_D_Gain' + '_' + CTAS], inplace = True) #Remove columns that would overfit the model
cols = df_lasso.columns.tolist() #Make list of columns names
n = int(cols.index('Invest' + '_' + CTAS)) #set n as a variable containing index of label column
cols = cols[:n] + cols[n+1:] + [cols[n]] #moves columns around so label column is last
df_lasso = df_lasso[cols] #Reassign with new column layout
X = df_lasso.iloc[:,:-1] #Create feature matrix
y = df_lasso.iloc[:,-1] #Create label vector
scaler = StandardScaler() #Instantiate standard scaler
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns) #Scale data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=20) #Split data into test/train sets

#Determine which alpha level is best for Logistic Regression with l1 penalty
# alphas= [.1, .25, .5,.75,1]
# for a in alphas:
#Instaniate and train model
clf_lr_l1 = LogisticRegression(penalty="l1", C = .1,solver='saga')
clf_lr_l1.fit(X_train,y_train)
#Remove redundant columns 
coeff_used = np.sum(clf_lr_l1.coef_!=0)
coeff_names = (clf_lr_l1.coef_!=0)
y_clf_pred_lasso = clf_lr_l1.predict(X_test) #Predict labels on test data
tn_lasso, fp_lasso, fn_lasso, tp_lasso = confusion_matrix(y_test, y_clf_pred_lasso).ravel() #Build confusion matrix 
print((tn_lasso + tp_lasso) / (tn_lasso+fp_lasso+fn_lasso+tp_lasso),coeff_used) #Look at accuracy and # of coefficents kept. Only 39 attributes remain

2419 1481
0.6384615384615384 39


C:\Users\grays\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [7]:
#View new master df with 39 features
df.iloc[:,coeff_names[0]].head(3) 

,Volume_CTAS,Month,Delta Change_CTAS,Volume_WAT,Delta Change_WAT,High Price_CTL,Open Price_CTL,Last Price_CTL,Yesterday_Close_CTL,%_Gain_MHK,...,Volume_GNW,%_Gain_GNW,Low Price_ABG,High Price_ABG,Volume_ABG,Delta Change_ABG,Volume_FWRD,%_Gain_FWRD,Volume_HFWA,Delta Change_^GSPC
2,1053048,1,-0.61,949600,-3.22,34.9,34.82,34.29,34.8,-1.1831,...,711300,-1.04439,13.61,13.9,34500,12.94,232922,-4.34783,3150,-14.03
3,1031141,1,-0.11,643100,-0.11,34.46,34.3,33.65,34.29,-1.082,...,623500,-1.90042,13.8,14.21,206500,-0.5,244866,-1.92614,54608,-4.31
4,1595734,1,0.41,938500,0.41,34.29,33.7,34.1,33.65,0.566251,...,765100,0.0379939,14.06,14.31,78100,0.01,236181,0.282818,12180,4.14


In [8]:
#Create lists and dictionaries 
accuracies_svm = []
accuracies_lr = []
accuracies_rf = []
accuracies_knn = []
svmd = {}
lrd = {}
rfd = {}
knnd = {}


In [9]:
#Create hyperparamter tuning options
param_grid_svm = [
  {'C': [.5, 1, 10], 'kernel': ['rbf']},
  {'C': [.5, 1, 10], 'kernel': ['poly'], 'degree': [2,3]},
 ]
param_grid_rf = {
    'n_estimators': [750, 1000],
    'max_depth': [5, 10,15,25]
}

param_grid_lr = {
    'C': [.01,.1,10,100]
}
param_grid_knn = {
    'n_neighbors': [25,50, 150, 300]
}

In [18]:
warnings.filterwarnings('once')
#Similar to lassso regression, however we are created 4 different models, cross validated 4 times trying to predict each of the 19 tickers
# by changing the threshold, we change the amount of 0/1 labels, which impacts the accuracy of the model. With almost 50/50 0/1 labels, the model predicts slightly over 50% of the time 
threshold = .3
for i in tickers:
    df_deep = deepcopy(df)
    df_deep['Next_Day_Close' + '_' + i] = df_deep['Last Price' + '_' + i].shift(-1)
    df_deep['Price Change' + '_' + i] = df_deep['Next_Day_Close' + '_' + i] - df_deep['Last Price' + '_' + i]
    df_deep['D_to_D_Gain' + '_' + i] = (df_deep['Price Change' + '_' + i] / df_deep['Last Price' + '_' + i])*100
    df_deep['Invest' + '_' + i] =  (df_deep['D_to_D_Gain' + '_' + i] > threshold).astype(int)
    df_deep.dropna(how = 'any',inplace = True)
    df_deep.drop(columns = ['Price Change' + '_' + i,'Next_Day_Close' + '_' + i,'D_to_D_Gain' + '_' + i], inplace = True)
    num_0 = df_deep.groupby('Invest' + '_' + i)['Low Price_CTAS'].count()[0] 
    num_1 = df_deep.groupby('Invest' + '_' + i)['Low Price_CTAS'].count()[1]
    cols = df_deep.columns.tolist()
    n = int(cols.index('Invest' + '_' + i))
    cols = cols[:n] + cols[n+1:] + [cols[n]]
    df_deep = df_deep[cols]
    X = df_deep.iloc[:,:-1]
    y = df_deep.iloc[:,-1]
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X.iloc[:,coeff_names[0]]), columns = X.iloc[:,coeff_names[0]].columns)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=20)
    
    #Utilize gridsearch to optimize our hyperparameters
    svmd[i]  = GridSearchCV(SVC(gamma  = 'auto'),param_grid_svm,cv =4).fit(X_train, y_train)
    lrd[i] = GridSearchCV(LogisticRegression(random_state=20, penalty = 'l1',solver = 'saga',max_iter=10000),param_grid_lr,cv=4).fit(X_train, y_train)
    rfd[i] = GridSearchCV(RandomForestClassifier(random_state=20),param_grid_rf,cv=4).fit(X_train,y_train)
    knnd[i]= GridSearchCV(KNeighborsClassifier(),param_grid_knn,cv=4).fit(X_train,y_train)
    
    #Assign each model to a dictionary 
    y_clf_pred_svm = svmd[i].predict(X_test)
    y_clf_pred_lr = lrd[i].predict(X_test)
    y_clf_pred_rf = rfd[i].predict(X_test)
    y_clf_pred_knn = knnd[i].predict(X_test)
    
    ##Determine accuracies for each model/ticker
    tn, fp, fn, tp = confusion_matrix(y_test, y_clf_pred_svm).ravel()
    tn1, fp1, fn1, tp1 = confusion_matrix(y_test, y_clf_pred_lr).ravel()
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, y_clf_pred_rf).ravel()
    tn3, fp3, fn3, tp3 = confusion_matrix(y_test, y_clf_pred_knn).ravel()
    
    #Append accuracies to lists to view predictive power
    accuracies_svm.append((tn + tp) / (tn+fp+fn+tp)) 
    accuracies_lr.append((tn1 + tp1) / (tn1+fp1+fn1+tp1)) 
    accuracies_rf.append((tn2 + tp2) / (tn2+fp2+fn2+tp2)) 
    accuracies_knn.append((tn3 + tp3) / (tn3+fp3+fn3+tp3))

In [19]:
#~60% accuracy for each model
print("Median Accuracy SVM:",statistics.median(accuracies_svm))
print("Median Accuracy Random Forest:",statistics.median(accuracies_rf))
print("Median Accuracy K_Nearest_Neighbor:",statistics.median(accuracies_knn))
print("Median Accuracy Logistic Regression:", statistics.median(accuracies_lr))

Median Accuracy SVM: 0.6
Median Accuracy Random Forest: 0.5974358974358974
Median Accuracy K_Nearest_Neighbor: 0.6012820512820513
Median Accuracy Logistic Regression: 0.5948717948717949


In [20]:
from datetime import datetime
currentMonth = datetime.now().month #Create month feature
currentYear = datetime.now().year #Create month feature
dfs = [] #Create list to maintains all intermediary dataframes
for i in tickers: #Go through each ticker and scrape data from yahoo finance
    lst_prices = [] #Create list to append all open/close/volume etc for repsective ticker (i)
    page = requests.get('https://finance.yahoo.com/quote/' + i +'/history') #Pull html from webage
    soup = BeautifulSoup(page.text, 'html.parser') #Parse html
    result = soup.find_all(['td'])[1:7] #Index first 7 since we only need most recent day
    prices = re.findall('[>](.*?)</span></td>',str(result)) #Use regex to parse data 
    for _ in range(0,len(prices)): #Skipping the "Adj Close"
        if _ == 4:
            pass
        else:
            lst_prices.append(float(prices[_].split(">",1)[1].replace(",",""))) #Add info to lists
    df_prices = pd.DataFrame([lst_prices]) #Create dataframe based on list
    #Rename and use feature generation to match inititial training dataset
    df_prices.rename(columns = {0:'Open Price' + '_' + i, 1:'High Price' + '_' + i, 2:'Low Price' + '_' + i, 3:'Last Price' + '_' + i, 4:'Volume' + '_' + i}, inplace = True)
    df_prices = df_prices[['Low Price' + '_' + i, 'High Price' + '_' + i, 'Open Price' + '_' + i, 'Volume' + '_' + i,'Last Price' + '_' + i]]
    df_prices['Year'] = currentYear
    df_prices['Month'] = currentMonth
    df_prices['Yesterday_Close' + '_' + i] = X.tail(1)['Last Price' + '_' + i].values[0]
    df_prices['Delta Change' + '_' + i] = df_prices['Last Price' + '_' + i] - df_prices['Yesterday_Close' + '_' + i]
    df_prices['%_Gain' + '_' + i] = (df_prices['Delta Change' + '_' + i] / df_prices['Last Price' + '_' + i])*100
    #Add each intermediary dataframe to list
    dfs.append(df_prices)
#Create master dataframe and use only the 39 features above
df_pred = pd.concat(dfs, sort=False, axis = 1)
df_pred = df_pred.loc[:,~df_pred.columns.duplicated()]
df_pred

,Low Price_CTAS,High Price_CTAS,Open Price_CTAS,Volume_CTAS,Last Price_CTAS,Year,Month,Yesterday_Close_CTAS,Delta Change_CTAS,%_Gain_CTAS,...,Delta Change_^DJI,%_Gain_^DJI,Low Price_^GSPC,High Price_^GSPC,Open Price_^GSPC,Volume_^GSPC,Last Price_^GSPC,Yesterday_Close_^GSPC,Delta Change_^GSPC,%_Gain_^GSPC
0,266.25,269.38,269.05,302300.0,268.36,2019,12,260.73,7.63,2.843196,...,226.25,0.794916,3216.57,3240.92,3240.09,3.013290e+09,3221.29,3191.45,29.84,0.926337


In [21]:
pred_knn = []
pred_svm = []
pred_log = []
#Make 0/1 predictions along with probability estimates for investment
#Probability that next day stock price will appreciate above "threshold (.3%)"
X_pred_scaled = pd.DataFrame(scaler.transform(df_pred.iloc[:,coeff_names[0]]))
for i in tickers:    
    pred_knn.append(round(knnd[i].predict_proba(X_pred_scaled)[0][1],2))
    pred_svm.append(round(rfd[i].predict_proba(X_pred_scaled)[0][1],2))
    pred_log.append(round(lrd[i].predict_proba(X_pred_scaled)[0][1],2))
    print("Ticker "+ i + " predictions","SVM:",svmd[i].predict(X_pred_scaled),"Log:",lrd[i].predict(X_pred_scaled),
          "Log Prob:",round(lrd[i].predict_proba(X_pred_scaled)[0][1],2),"RF:",rfd[i].predict(X_pred_scaled),
          "RF Prob:",round(rfd[i].predict_proba(X_pred_scaled)[0][1],2),"KNN:",knnd[i].predict(X_pred_scaled),
          "KNN Prob:",round(knnd[i].predict_proba(X_pred_scaled)[0][1],2))

Ticker CTAS predictions SVM: [1] Log: [0] Log Prob: 0.4 RF: [0] RF Prob: 0.43 KNN: [0] KNN Prob: 0.4
Ticker WAT predictions SVM: [1] Log: [0] Log Prob: 0.4 RF: [0] RF Prob: 0.43 KNN: [0] KNN Prob: 0.4
Ticker CTL predictions SVM: [0] Log: [0] Log Prob: 0.39 RF: [0] RF Prob: 0.39 KNN: [0] KNN Prob: 0.43
Ticker IT predictions SVM: [1] Log: [0] Log Prob: 0.42 RF: [0] RF Prob: 0.36 KNN: [0] KNN Prob: 0.41
Ticker MHK predictions SVM: [0] Log: [0] Log Prob: 0.4 RF: [0] RF Prob: 0.41 KNN: [0] KNN Prob: 0.37
Ticker ALK predictions SVM: [1] Log: [0] Log Prob: 0.44 RF: [0] RF Prob: 0.41 KNN: [0] KNN Prob: 0.38
Ticker EEFT predictions SVM: [0] Log: [0] Log Prob: 0.42 RF: [0] RF Prob: 0.38 KNN: [0] KNN Prob: 0.41
Ticker CACC predictions SVM: [0] Log: [0] Log Prob: 0.42 RF: [0] RF Prob: 0.41 KNN: [0] KNN Prob: 0.4
Ticker MTG predictions SVM: [0] Log: [0] Log Prob: 0.43 RF: [0] RF Prob: 0.45 KNN: [0] KNN Prob: 0.44
Ticker LSTR predictions SVM: [0] Log: [0] Log Prob: 0.42 RF: [0] RF Prob: 0.36 KNN: [0